In [1]:
import numpy as np
import pandas as pd
import re
from pprint import pprint
clinical_df = pd.read_excel("20200414.xlsx", sheet_name=0)
library_df = pd.read_excel("20200414.xlsx", sheet_name=1)

In [2]:
sampleId_count_clinical = clinical_df["样本编号"].value_counts()
sampleId_unique_clinical = sampleId_count_clinical.index[sampleId_count_clinical==1]
print("total sampleId in clinical_df: {}; unique sampleId(count=1) in clinical_df: {}".format(clinical_df.shape[0], sampleId_unique_clinical.shape[0]))

total sampleId in clinical_df: 2357; unique sampleId(count=1) in clinical_df: 2065


In [3]:
library_df.loc[:, "虚拟入库样本名"] = library_df.loc[:, "虚拟入库样本名"].fillna(method='ffill')
library_df.loc[:, "pooling比例"] = library_df.loc[:, "pooling比例"].fillna(method='ffill')	
library_df.loc[:, "取样ng"] = library_df.loc[:, "取样ng"].fillna(method='ffill')	
library_df.loc[:, "体积ul"] = library_df.loc[:, "体积ul"].fillna(method='ffill')	
library_df.loc[:, "杂交探针"] = library_df.loc[:, "杂交探针"].fillna(method='ffill')	
library_df.loc[:, "杂交时间h"] = library_df.loc[:, "杂交时间h"].fillna(method='ffill')	
library_df.loc[:, "PostPCR循环数"] = library_df.loc[:, "PostPCR循环数"].fillna(method='ffill')	
library_df.loc[:, "PostPCR浓度ng/ul"] = library_df.loc[:, "PostPCR浓度ng/ul"].fillna(method='ffill')	
library_df.loc[:, "洗脱体积"] = library_df.loc[:, "洗脱体积"].fillna(method='ffill')	
library_df.loc[:, "文库质控"] = library_df.loc[:, "文库质控"].fillna(method='ffill')	
library_df.loc[:, "文库名称"] = library_df.loc[:, "文库名称"].fillna(method='ffill')
library_df = library_df.fillna('无')
library_df["华大样本编号P"] = library_df["华大样本编号"].str.slice_replace(2, 3, repl='P')
tmp = [ str(int(x)) if x!="无" else x for x in library_df.loc[:,"建库日期"].to_list()]
library_df["建库日期"] = [ x[:4]+"/"+x[4:6]+"/"+x[6:] if x!="无" else "" for x in  tmp]
sampleId_count_libaray = library_df["华大样本编号P"].value_counts()
sampleId_unique_libaray = sampleId_count_libaray.index[sampleId_count_libaray==1]
print("total sampleId in library_df: {}; unique sampleId(count=1) in library_df: {}".format(library_df.shape[0], sampleId_unique_libaray.shape[0]))

total sampleId in library_df: 2197; unique sampleId(count=1) in library_df: 1408


In [4]:
sampleId_unique_overlap = sampleId_unique_libaray[sampleId_unique_libaray.isin(sampleId_unique_clinical)]
print("overlap unique sampleId(count=1) in clinical_df and library_df: {}".format(sampleId_unique_overlap.shape[0]))

overlap unique sampleId(count=1) in clinical_df and library_df: 1105


In [5]:
library_df_filt = library_df.loc[library_df["华大样本编号P"].isin(sampleId_unique_overlap)]
library_df_filt["建库编号"] = library_df["华大样本编号"].str.replace(re.compile(r'$'), '_1_m1')
library_df_filt["DNA提取编号"] = library_df["华大样本编号"].str.replace(re.compile(r'$'), '_1')
count = {}
libraryId_list = []
for dnaId in library_df_filt.loc[:, "DNA提取编号"]:
    dnaId = str(dnaId)
    count[dnaId] = count[dnaId]+1 if dnaId in count else 1
    libraryId_list.append("{}_m{}".format(dnaId, count[dnaId]))


clinical_df_filt = clinical_df.loc[clinical_df["样本编号"].isin(sampleId_unique_overlap)]


In [6]:
library_df_filt.loc[:, "文库名称"] = library_df_filt.loc[:, "文库名称"].str.strip()
library_df_filt.loc[:, "捕获文库操作人"] = library_df_filt.loc[:, "文库名称"].str.extract(r'（(.+)）', flags=re.IGNORECASE, expand=True).iloc[:,0]

In [7]:
captureId = [ str(x) for x in library_df_filt.loc[:, "文库名称"]]
# print("before:{}".format(captureId[:30]))
# print("before:{}".format(captureId[-30:]))
# print(">>>>>>>>>")
for idx in range(len(captureId)):
    captureId[idx] = re.sub(r'（\w+）', "", captureId[idx])
    captureId[idx] = re.sub(r'-cpg.*?-', r'-CpGiant-', captureId[idx], flags=re.IGNORECASE)
    captureId[idx] = re.sub(r'-(\d{6})$', r'-20\1', str(captureId[idx]))
    captureId[idx] = re.sub(r'-(\d{6})-', r'-20\1-', str(captureId[idx]))
    captureId[idx] = re.sub(r'ssM-(\d+)-CpGiant-(\d{8})', r'ssM-CpGiant-\1-\2', captureId[idx])
    captureId[idx] = re.sub(r'ssM-CpGiant-(\d{8})-(\d+)', r'ssM-CpGiant-\2-\1', captureId[idx])
# print("after:{}".format(captureId[:30]))
# print("after:{}".format(captureId[-30:]))

In [8]:
year_pre = int(re.match(r'^.+-(\d{4})\d{4}$', captureId[0]).group(1))
for idx in range(len(captureId)):
    try:
        year = int(re.match(r'^.+-(\d{4})\d{4}$', captureId[idx]).group(1))
        if year<year_pre:
            captureId[idx] = re.sub(r'^(.+)-\d{4}(\d{4})$', r'\1-'+str(year_pre)+r'\2', captureId[idx])
        else:
            year_pre = year
    except:
        pass
# captureId

library_df_filt.loc[:, "文库名称"] = captureId
library_df_filt.loc[:, "杂交日期"] = [ re.sub(r'^.+-(\d{4})(\d{2})(\d{2})$', r"\1/\2/\3", x) if x!='-' else '-' for x in captureId]

In [9]:
with pd.ExcelWriter("library.20200424.xlsx") as writer:
    library_df_filt.to_excel(writer, sheet_name="建库表(clinicalDf_overlap_libraryDf)", index=False)
    clinical_df_filt.to_excel(writer, sheet_name="临床信息表(clinicalDf_overlap_libraryDf)", index=False)


In [10]:
library_df_filt.head

<bound method NDFrame.head of                  建库编号       DNA提取编号 管上编号    是否临床        文库名 样本标签 index列表  \
226   18S4146706_1_m1  18S4146706_1    1  Clinic      ssM-1    P   57,58   
227   18S4146707_1_m1  18S4146707_1    2  Clinic      ssM-2    P   59,60   
228   18S4146708_1_m1  18S4146708_1    3  Clinic      ssM-3    P   61,62   
229   18S4146709_1_m1  18S4146709_1    4  Clinic      ssM-4    P   63,64   
230   18S4146710_1_m1  18S4146710_1    5  Clinic      ssM-5    P   65,66   
...               ...           ...  ...     ...        ...  ...     ...   
2192  20L5383633_1_m1  20L5383633_1   12  Clinic  ssM-12-pl    T      68   
2193  20S5383594_1_m1  20S5383594_1   13  Clinic  ssM-13-pl    T      69   
2194  20L5383634_1_m1  20L5383634_1   14  Clinic  ssM-14-pl    T      70   
2195  20S5383595_1_m1  20S5383595_1   15  Clinic  ssM-15-pl    T      71   
2196  20L5383635_1_m1  20L5383635_1   16  Clinic  ssM-16-pl    T      72   

            建库日期 建库方法    试剂批次  ...                       